In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import datetime
import calendar
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import plotly.graph_objects as go
import numpy as np

In [ ]:
# Import dataset
df = pd.read_csv(os.path.join(os.getcwd(), "data", "bank-dataset.csv"))
df.shape

In [ ]:
# Check month-day validity (suppose year is 2021) 

def date_is_valid(day, month, year=2021):
    # This function check if a date is valid
    try: 
        datetime.date(year, month, day)
    except:
        return False
    return True

calendar_dict = {month.lower(): index for index, month in enumerate(calendar.month_abbr) if month} # mapping calendar short name to ordinal month number
df['month'].replace(calendar_dict, inplace=True) # replacing month name with numeric values
df['date_is_valid'] = df[['day','month']].apply(lambda x: date_is_valid(x[0],x[1]), axis=1) # check for validity
print(f"There are {sum(df['date_is_valid']==False)} invalid dates")

df = df[df['date_is_valid']==True] # Keep valid dates
df.drop(columns=['date_is_valid'], inplace=True) # Valid date column is useless from now on

# Instead of keeping month and day, I want to have the number of days since beginning of the year
df['last_contact_day'] = df[['day','month']].apply(lambda x: (datetime.date(2021, x[1],x[0]) - datetime.date(2021, 1,1)).days, axis=1)
df.drop(columns=['day','month'], inplace=True)

In [ ]:
# Uniformy the marital column. There are 3 possible category whose name start with a different letter. The fastest way to uniformy the value in this case is to take the initial letter.
df['marital'] = df['marital'].apply(lambda x: x[0].lower())

In [ ]:
# Check for invalid duration vaues
invalid_duration = (df['duration']==0).sum()
print(f"There are {invalid_duration} invalid duration values. Drop them")

In [ ]:
# Check unbalanced target variable
le = LabelEncoder()
df["y"] = le.fit_transform(df["y"])
df['y'].value_counts() / len(df)

# Remove target from dataset
y = df["y"]
df.drop(columns=["y"], inplace=True)

In [ ]:
# Select numerical and categorical features
numerical_features = ["age", "balance", "duration", "campaign", "pdays", "previous","last_contact_day"]
categorical_features = [col for col in df.columns if col not in numerical_features]

In [ ]:
# Check for correlated numerical features
df_corr = df[numerical_features].corr() 
fig = px.imshow(df_corr, aspect='auto', color_continuous_scale='Reds')
fig.layout.height = 600
fig.layout.width = 600
fig.show()

In [ ]:
# BoxPlot for outlier
fig = px.box(df[numerical_features].melt(), y='value', facet_col='variable').update_yaxes(matches=None)
fig.show()

In [ ]:
# Split train and tes
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=0, stratify=y)

In [ ]:
# Check for nan
X_train.isna().sum()

In [ ]:
# To treat outlier, set them to nan and proced with and missing value inputation (since we have already other nan to impute)
X_train.loc[X_train['age'] > 100,'age'] = np.nan
X_train.loc[X_train['age'] < 18, 'age'] = np.nan
X_train.loc[X_train['previous'] > 20, 'previous'] = np.nan

In [ ]:
# Value imputation: using median for numeric feature (some feateures have skewed distribution) and mode for categorial
imputer_numeric = SimpleImputer(strategy='median')
imputer_categoric = SimpleImputer(strategy='most_frequent')

imputer_numeric.fit(X_train[numerical_features])
imputer_categoric.fit(X_train[categorical_features])


X_train[numerical_features] = imputer_numeric.transform(X_train[numerical_features])
X_test[numerical_features] = imputer_numeric.transform(X_test[numerical_features])
X_train[categorical_features] = imputer_categoric.transform(X_train[categorical_features])
X_test[categorical_features] = imputer_categoric.transform(X_test[categorical_features])


In [ ]:
# BoxPlot for outlier
fig = px.box(X_train[numerical_features].melt(), y='value', facet_col='variable').update_yaxes(matches=None)
fig.show()

In [ ]:
# Save dateset to use in different models
X_train.to_csv(os.path.join(os.getcwd(), "data", "X_train.csv"), index=False)
X_test.to_csv(os.path.join(os.getcwd(), "data", "X_test.csv"), index=False)
y_train.to_csv(os.path.join(os.getcwd(), "data", "y_train.csv"), index=False)
y_test.to_csv(os.path.join(os.getcwd(), "data", "y_test.csv"), index=False)